# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [6]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/Users/roytelles/Desktop/Python Codes/UdacityStuff/DataEng Nano/Project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event
# data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be
# used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7],
                         row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [141]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [142]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [143]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
# Because we want to do three different queries, we will need different tables that
# partition the data differently

# Query 1:
#     Table Name: Music library
#         Column 1: sessionId text,
#         Column 2: itemInSession text,
#         Column 3: artist text,
#         Column 4: song text,
#         Column 5: length text,
#         PRIMARY KEY (sessionId, itemInSession)

# Query 2:
#     Table Name: Artist library
#         Column 1: userId text,
#         Column 2: sessionId text,
#         Column 3: itemInSession text,
#         Column 4: artist text,
#         Column 5: song text,
#         Column 6: firstName text,
#         Column 7: lastName text,
#         PRIMARY KEY (userId, sessionId, itemInSession)

# Query 3:
#     Table Name: User library
#         Column 1: userId text,
#         Column 2: firstName text,
#         Column 3: lastName text,
#         Column 4: song text,
#         PRIMARY KEY (song, userId, firstName, lastName)

In [69]:
drop_music_library = "DROP TABLE music_library;"

try:
    session.execute(drop_music_library)
except Exception as e:
    print(e)

In [144]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app
## history that was heard during sessionId = 338, and itemInSession = 4
query = """
    CREATE TABLE IF NOT EXISTS music_library (
        sessionId text,
        itemInSession text,
        artist text,
        song text,
        length text,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [145]:
# We have provided part of the code to set up the CSV file. Please complete the Apache
# Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO music_library (sessionId, itemInSession, artist, song, length)
        """
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[3], line[8], line[0], line[9], line[5]))
    print('Values inserted')

Values inserted


#### Do a SELECT to verify that the data have been inserted into each table

In [146]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
verify = """
    SELECT * FROM music_library WHERE iteminsession = '338' AND sessionid = '4';
"""

try:
    rows = session.execute(verify)
except Exception as e:
    print(e)
    
for row in rows:
    print(f"""Artist: {row.artist}
        \nSong: {row.song}
        \nLength: {row.length}
    """)

Artist: Faithless
        
Song: Music Matters (Mark Knight Dub)
        
Length: 495.3073
    


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [112]:
drop_artist_library = "DROP TABLE artist_library;"

try:
    session.execute(drop_artist_library)
except Exception as e:
    print(e)

In [147]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by
## itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = """
    CREATE TABLE IF NOT EXISTS artist_library (
        userId text,
        sessionId text,
        itemInSession text,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
"""

try:
    session.execute(query)
    print('Table created')
except Exception as e:
    print(e)

Table created


In [148]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO artist_library
                (userId, sessionId, itemInSession, artist, song, firstName, lastName)
        """
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))
    print('Values inserted')

Values inserted


In [149]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
verify = """
    SELECT * FROM artist_library
    WHERE userid = '10' AND sessionid = '182'
    ORDER BY sessionid, iteminsession;
"""

try:
    rows = session.execute(verify)
except Exception as e:
    print(e)
    
for row in rows:
    print(f"""Artist: {row.artist}
        \nSong: {row.song}
        \nUser: {row.firstname} {row.lastname}\n
    """)

Artist: Down To The Bone
        
Song: Keep On Keepin' On
        
User: Sylvie Cruz

    
Artist: Three Drives
        
Song: Greece 2000
        
User: Sylvie Cruz

    
Artist: Sebastien Tellier
        
Song: Kilometer
        
User: Sylvie Cruz

    
Artist: Lonnie Gordon
        
Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
        
User: Sylvie Cruz

    


In [130]:
drop_user_library = "DROP TABLE user_library;"

try:
    session.execute(drop_user_library)
except Exception as e:
    print(e)

In [150]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    CREATE TABLE IF NOT EXISTS user_library (
        userId text,
        firstName text,
        lastName text,
        song text,
        PRIMARY KEY (song, userId, firstName, lastName)
    )
"""

try:
    session.execute(query)
    print('Table created')
except Exception as e:
    print(e)

Table created


In [151]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO user_library
                (userId, firstName, lastName, song)
        """
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[10], line[1], line[4], line[9]))
    print('Values inserted')

Values inserted


In [152]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
verify = """
    SELECT * FROM user_library
    WHERE song = 'All Hands Against His Own';
"""

try:
    rows = session.execute(verify)
except Exception as e:
    print(e)

print("Users:\n")
for row in rows:
    print(f"{row.firstname} {row.lastname}\n")

Users:

Jacqueline Lynch

Tegan Levine

Sara Johnson



### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [153]:
drop_music_library = "DROP TABLE music_library;"
drop_artist_library = "DROP TABLE artist_library;"
drop_user_library = "DROP TABLE user_library;"

drop_queries = [drop_music_library, drop_artist_library, drop_user_library]

try:
    for query in drop_queries:
        session.execute(query)
    print("Tables dropped")
except Exception as e:
    print(e)

Tables dropped


### Close the session and cluster connection¶

In [154]:
session.shutdown()
cluster.shutdown()